In [3]:
MODEL_TYPE='protenix'
VALIDATION=False

In [ ]:
!pip show biopython


## Install requirements 

RNA 3D 예측 결과와 실제 구조를 비교하고, 이를 평가하는 파이프 라인.

In [4]:
if MODEL_TYPE=='protenix' and VALIDATION:
    !pip install --no-deps protenix
    !pip install biopython
    !pip install ml-collections
    !pip install biotite==1.0.1
    !pip install rdkit
!export PROTENIX_DATA_ROOT_DIR=/kaggle/input/protenix-checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.1/441.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.4 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
protenix 0.4.6 requires biotite==1.0.1, which is not installed.
protenix 0.4.6 requires deepspeed>=0.15.1, which is not installed.
protenix 0.4.6 requires icecream, which is not installed.
protenix 0.4.6 requires ipdb, which is not installed.
protenix 0.4.6 requires ml_collections, which is not installed.
protenix 0.4.6 requires modelcif==0.7, which is not installed.
protenix 0.4.6 requires py3Dmol, which is not installed.
protenix 0.4.6 requires rdkit, which is not installed.
protenix 0.4.6 requires scikit-learn-extra, which is not installed.
protenix 0.4.6 requires biopython==1.83, but you have biopython 1.85 which is incompatible.
protenix 0.4.6 requ

In [5]:
! mkdir /af3-dev 
! ln -s /kaggle/input/protenix-checkpoints /af3-dev/release_data
! ls /af3-dev/release_data/

components.v20240608.cif		model_v0.2.0.pt
components.v20240608.cif.rdkit_mol.pkl


## Helper scripts

In [6]:
import Bio

from copy import deepcopy

import pandas as pd
from Bio.PDB import Atom, Model, Chain, Residue, Structure, PDBParser
from Bio import SeqIO
import os, sys
import re
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
time0=time.time()
from Bio import AlignIO
print('IMPORT OK !!!!')

IMPORT OK !!!!


In [17]:
PYTHON = sys.executable
print('PYTHON',PYTHON)

RHONET_DIR=\
'/kaggle/input/data-for-demo-for-rhofold-plus-with-kaggle-msa/RhoFold-main'
#'<your downloaded rhofold repo>/RhoFold-main'

USALIGN = \
'/kaggle/working//USalign'
#'<your us align path>/USalign'

os.system('cp /kaggle/input/usalign/USalign /kaggle/working/')
os.system('sudo chmod u+x /kaggle/working//USalign')
sys.path.append(RHONET_DIR)


DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'


# helper ----
# 딕셔너리 <-> 키 매핑
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

# visualisation helper ----
def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)




# xyz df helper --------------------
def get_truth_df(target_id):
    truth_df = LABEL_DF[LABEL_DF['target_id'] == target_id]
    truth_df = truth_df.reset_index(drop=True)
    return truth_df

def parse_output_to_df(output, seq, target_id):
    df = []
    chain_data = []
    for i, res in enumerate(seq):
        d=dict(ID = target_id,
                    resname=res,
                    resid=i+1)
        for n in range(len(output)):
            d={**d, f'x_{n+1}': round(output[n,i,0].item(),3),
                     f'y_{n+1}': round(output[n,i,1].item(),3),
                     f'z_{n+1}': round(output[n,i,2].item(),3)}
        chain_data.append(d)

    if len(chain_data)!=0:
        chain_df = pd.DataFrame(chain_data)
        df.append(chain_df)
        ##print(chain_df)
    return df

def parse_pdb_to_df(pdb_file, target_id):
    parser = PDBParser()
    structure = parser.get_structure('', pdb_file)

    df = []
    for model in structure:
        for chain in model:
            print(chain)
            chain_data = []
            for residue in chain:
                # print(residue)
                if residue.get_resname() in ['A', 'U', 'G', 'C']:
                    # Check if the residue has a C1' atom
                    if 'C1\'' in residue:
                        atom = residue['C1\'']
                        xyz = atom.get_coord()
                        resname = residue.get_resname()
                        resid = residue.get_id()[1]

                        #todo detect discontinous: resid = prev_resid+1
                        #ID	resname	resid	x_1	y_1	z_1
                        chain_data.append(dict(
                            ID = target_id+'_'+str(resid),
                            resname=resname,
                            resid=resid,
                            x_1=xyz[0],
                            y_1=xyz[1],
                            z_1=xyz[2],
                        ))
                        ##print(f"Residue {resname} {resid}, Atom: {atom.get_name()}, xyz: {xyz}")

            if len(chain_data)!=0:
                chain_df = pd.DataFrame(chain_data)
                df.append(chain_df)
                ##print(chain_df)
    return df

# usalign helper --------------------
def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
):
    """
    Writes a single line of PDB format based on provided atom information.

    Args:
        atom_name (str): Name of the atom (e.g., "N", "CA").
        atom_serial (int): Atom serial number.
        residue_name (str): Residue name (e.g., "ALA").
        chain_id (str): Chain identifier.
        residue_num (int): Residue number.
        x_coord (float): X coordinate.
        y_coord (float): Y coordinate.
        z_coord (float): Z coordinate.
        occupancy (float, optional): Occupancy value (default: 1.0).
        b_factor (float, optional): B-factor value (default: 0.0).

    Returns:
        str: A single line of PDB string.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'

def write_xyz_to_pdb(df, pdb_file, xyz_id = 1):
    resolved_cnt = 0
    with open(pdb_file, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

def parse_usalign_for_tm_score(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)

def parse_usalign_for_transform(output):
    # Locate the rotation matrix section
    matrix_lines = []
    found_matrix = False

    for line in output.splitlines():
        if "The rotation matrix to rotate Structure_1 to Structure_2" in line:
            found_matrix = True
        elif found_matrix and re.match(r'^\d+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+$', line):
            matrix_lines.append(line)
        elif found_matrix and not line.strip():
            break  # Stop parsing if an empty line is encountered after the matrix

    # Parse the rotation matrix values
    rotation_matrix = []
    for line in matrix_lines:
        parts = line.split()
        row_values = list(map(float, parts[1:]))  # Skip the first column (index)
        rotation_matrix.append(row_values)

    return np.array(rotation_matrix)

def call_usalign(predict_df, truth_df, verbose=1):
    truth_pdb = '~truth.pdb'
    predict_pdb = '~predict.pdb'
    write_xyz_to_pdb(predict_df, predict_pdb, xyz_id=1)
    write_xyz_to_pdb(truth_df, truth_pdb, xyz_id=1)

    command = f'{USALIGN} {predict_pdb} {truth_pdb} -atom " C1\'" -m -'
    output = os.popen(command).read()
    if verbose==1:
        print(output)
    tm_score = parse_usalign_for_tm_score(output)
    transform = parse_usalign_for_transform(output)
    return tm_score, transform

print('HELPER OK!!!')

PYTHON /usr/bin/python3
HELPER OK!!!


In [25]:
if MODEL_TYPE=='protenix':
    
    
    from runner.batch_inference import get_default_runner
    from runner.inference import update_inference_configs, InferenceRunner

    from protenix.data.infer_data_pipeline import InferenceDataset

    np.random.seed(0)
    torch.random.manual_seed(0)
    torch.cuda.manual_seed_all(0)

class DictDataset(InferenceDataset):
    def __init__(
        self,
        seq_list: list,
        dump_dir: str,
        id_list: list = None,
        msa_root_1: str = "/kaggle/input/stanford-rna-3d-folding/MSA",
        msa_root_2: str = "/kaggle/input/stanford-rna-3d-folding/MSA_v2",
    ) -> None:
        self.dump_dir = dump_dir
        self.msa_roots = [msa_root_2, msa_root_1]  # 우선순위: v2 > v1
        self.use_msa = True  # Protenix 내부에서 참조하므로 반드시 필요

        if isinstance(id_list, type(None)):
            self.inputs = [{"sequences": [{"rnaSequence": {"sequence": seq, "count": 1}}], "name": "query"}
                           for seq in seq_list]
        else:
            self.inputs = [{"sequences": [{"rnaSequence": {"sequence": seq, "count": 1}}], "name": i}
                           for i, seq in zip(id_list, seq_list)]

    def __getitem__(self, index):
        data, atom_array, error_msg = super().__getitem__(index)
        target_id = self.inputs[index]["name"]

        msa_tensor = None
        for root in self.msa_roots:
            msa_path = os.path.join(root, f"{target_id}.MSA.fasta")
            if os.path.exists(msa_path):
                try:
                    msa_tensor = self.load_msa_fasta(msa_path)
                    print(f"✅ Loaded MSA for {target_id} from {root} | shape={msa_tensor.shape}")
                    break
                except Exception as e:
                    print(f"⚠️ Error loading MSA for {target_id}: {e}")

        if msa_tensor is not None and msa_tensor.shape[0] >= 10:
            if "input_feature_dict" not in data:
                data["input_feature_dict"] = {}
            data["input_feature_dict"]["msa_feat"] = msa_tensor
            print(f"✅ Injected msa_feat for {target_id} | shape={msa_tensor.shape}")
        else:
            print(f"⚠️ Skipped msa_feat for {target_id} | depth insufficient or not found")

        return data, atom_array, error_msg

    def load_msa_fasta(self, msa_path):
        from Bio import AlignIO
        alignment = AlignIO.read(msa_path, "fasta")
        alignment = alignment[:64]  # optional: top 64 rows only
        L = len(alignment[0])
        N = len(alignment)

        # RNA base vocabulary mapping
        alphabet = {'A': 0, 'C': 1, 'G': 2, 'U': 3, '-': 4}
        msa_tensor = torch.zeros((N, L), dtype=torch.long)
        for i, record in enumerate(alignment):
            msa_tensor[i] = torch.tensor([
                alphabet.get(residue.upper(), 4)  # default to gap if unknown
                for residue in record.seq
            ])
        return msa_tensor


In [9]:
if MODEL_TYPE=='protenix':

    from configs.configs_base import configs as configs_base
    from configs.configs_data import data_configs
    from configs.configs_inference import inference_configs
    from protenix.config.config import parse_configs

    configs_base["use_deepspeed_evo_attention"] = (
    os.environ.get("USE_DEEPSPEED_EVO_ATTENTION", False) == "true")
    configs_base["model"]["N_cycle"] = 10 #10
    configs_base["sample_diffusion"]["N_sample"] = (1 if VALIDATION else 5)
    configs_base["sample_diffusion"]["N_step"] = 200
    inference_configs['load_checkpoint_path']='/kaggle/input/protenix-checkpoints/model_v0.2.0.pt'
    configs = {**configs_base, **{"data": data_configs}, **inference_configs}

    configs = parse_configs(
            configs=configs,
            fill_required_with_null=True,
        )
    
    runner=InferenceRunner(configs)
    # print(type(runner.model))          # 클래스 이름
    # print(runner.model)                # 전체 구조 요약

train scheduler 16.0
inference scheduler 16.0
Diffusion Module has 16.0


/usr/local/lib/python3.10/dist-packages/runner/inference.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, self.device)


<class 'protenix.model.protenix.Protenix'>
Protenix(
  (input_embedder): InputFeatureEmbedder(
    (atom_attention_encoder): AtomAttentionEncoder(
      (linear_no_bias_f): Linear(in_features=389, out_features=128, bias=False)
      (linear_no_bias_d): Linear(in_features=3, out_features=16, bias=False)
      (linear_no_bias_invd): Linear(in_features=1, out_features=16, bias=False)
      (linear_no_bias_v): Linear(in_features=1, out_features=16, bias=False)
      (linear_no_bias_cl): Linear(in_features=128, out_features=16, bias=False)
      (linear_no_bias_cm): Linear(in_features=128, out_features=16, bias=False)
      (small_mlp): Sequential(
        (0): ReLU()
        (1): Linear(in_features=16, out_features=16, bias=False)
        (2): ReLU()
        (3): Linear(in_features=16, out_features=16, bias=False)
        (4): ReLU()
        (5): Linear(in_features=16, out_features=16, bias=False)
      )
      (atom_transformer): AtomTransformer(
        (diffusion_transformer): Diffusion

In [16]:
if VALIDATION:
    LABEL_DF = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')
    LABEL_DF['target_id'] = LABEL_DF['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))
    train_df=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')


In [ ]:
if MODEL_TYPE == 'protenix' and VALIDATION:
    import warnings
    warnings.filterwarnings("ignore")  

    train_df['protenix_tm_score'] = None
    dataset = DictDataset(
        seq_list=train_df.sequence,
        dump_dir='output',
        id_list=train_df.target_id,
        msa_root_1="/kaggle/input/stanford-rna-3d-folding/MSA",
        msa_root_2="/kaggle/input/stanford-rna-3d-folding/MSA_v2",
    )
    num_data = len(dataset)

    for i, seq in tqdm(enumerate(train_df.sequence), total=num_data):
        target_id = train_df.loc[i, 'target_id']
        print(f"\n▶ {i}: {target_id} | len={len(seq)} | TM={train_df.loc[i, 'protenix_tm_score']}")

        if not pd.isna(train_df.loc[i, 'protenix_tm_score']):
            print("⏭️ TM-score already exists → skip")
            continue
        if len(seq) > 300:
            print("⏭️ Sequence too long → skip")
            continue

        truth_df = get_truth_df(target_id)
        num_valid_coords = sum(~np.isnan(truth_df.x_1))
        if num_valid_coords < 3:
            print("⏭️ Not enough ground-truth coordinates → skip")
            continue

        data, atom_array, data_error_message = dataset[i]
        if data_error_message != '':
            print(f"⏭️ Error loading data: {data_error_message}")
            continue

        print("📦 input keys:", data['input_feature_dict'].keys())
        if "msa_feat" in data["input_feature_dict"]:
            msa_feat = data["input_feature_dict"]["msa_feat"]
            print("✅ msa_feat shape:", msa_feat.shape)
        else:
            print("❌ msa_feat not present")

        new_configs = update_inference_configs(configs, data["N_token"].item())
        runner.update_model_configs(new_configs)

        prediction = runner.predict(data)
        prediction = prediction['coordinate'][:, data['input_feature_dict']['atom_to_tokatom_idx'] == 12]

        print(f"🧪 Prediction shape: {prediction.shape}, contains NaN: {torch.isnan(prediction).any().item()}")

        result = parse_output_to_df(prediction[:1], seq, target_id)[0]

        try:
            tm_score, transform = call_usalign(result, truth_df, verbose=0)
            train_df.loc[i, 'protenix_tm_score'] = tm_score
            print(f"✅ TM-score: {tm_score:.4f}")
        except Exception as e:
            print(f"❌ TM-score 계산 실패 (target_id={target_id}): {e}")

        if (time.time() - time0) > (12 * 3600 - 360):
            print("⏹️ 시간 제한 도달 → 종료")
            break

    train_df.to_csv('tm_scores.csv', index=False)
    print("📊 평균 TM-score:", train_df.protenix_tm_score.mean())
    display(train_df.protenix_tm_score.hist())


  0%|          | 0/844 [00:00<?, ?it/s]


▶ 0: 1SCL_A | len=29 | TM=None
✅ Loaded MSA for 1SCL_A from /kaggle/input/stanford-rna-3d-folding/MSA_v2 | shape=torch.Size([64, 29])
✅ Injected msa_feat for 1SCL_A | shape=torch.Size([64, 29])
📦 input keys: dict_keys(['token_index', 'residue_index', 'asym_id', 'entity_id', 'sym_id', 'restype', 'token_bonds', 'ref_pos', 'ref_mask', 'ref_element', 'ref_charge', 'ref_atom_name_chars', 'ref_space_uid', 'has_frame', 'frame_atom_index', 'atom_to_token_idx', 'atom_to_tokatom_idx', 'is_protein', 'is_ligand', 'is_dna', 'is_rna', 'resolution', 'mol_id', 'mol_atom_index', 'entity_mol_id', 'pae_rep_atom_mask', 'plddt_m_rep_atom_mask', 'distogram_rep_atom_mask', 'modified_res_mask', 'bond_mask', 'msa', 'has_deletion', 'deletion_value', 'profile', 'deletion_mean', 'prot_pair_num_alignments', 'prot_unpair_num_alignments', 'rna_pair_num_alignments', 'rna_unpair_num_alignments', 'template_restype', 'template_all_atom_mask', 'template_all_atom_positions', 'msa_feat'])
✅ msa_feat shape: torch.Size([64,

  0%|          | 1/844 [00:13<3:15:35, 13.92s/it]

🧪 Prediction shape: torch.Size([1, 29, 3]), contains NaN: False
✅ TM-score: 0.2296

▶ 1: 1RNK_A | len=34 | TM=None
✅ Loaded MSA for 1RNK_A from /kaggle/input/stanford-rna-3d-folding/MSA_v2 | shape=torch.Size([10, 34])
✅ Injected msa_feat for 1RNK_A | shape=torch.Size([10, 34])
📦 input keys: dict_keys(['token_index', 'residue_index', 'asym_id', 'entity_id', 'sym_id', 'restype', 'token_bonds', 'ref_pos', 'ref_mask', 'ref_element', 'ref_charge', 'ref_atom_name_chars', 'ref_space_uid', 'has_frame', 'frame_atom_index', 'atom_to_token_idx', 'atom_to_tokatom_idx', 'is_protein', 'is_ligand', 'is_dna', 'is_rna', 'resolution', 'mol_id', 'mol_atom_index', 'entity_mol_id', 'pae_rep_atom_mask', 'plddt_m_rep_atom_mask', 'distogram_rep_atom_mask', 'modified_res_mask', 'bond_mask', 'msa', 'has_deletion', 'deletion_value', 'profile', 'deletion_mean', 'prot_pair_num_alignments', 'prot_unpair_num_alignments', 'rna_pair_num_alignments', 'rna_unpair_num_alignments', 'template_restype', 'template_all_atom_m

  0%|          | 2/844 [00:27<3:16:04, 13.97s/it]

...

🧪 Prediction shape: torch.Size([1, 21, 3]), contains NaN: False
✅ TM-score: 0.3800

▶ 242: 2O33_A | len=20 | TM=None
✅ Loaded MSA for 2O33_A from /kaggle/input/stanford-rna-3d-folding/MSA_v2 | shape=torch.Size([26, 20])
✅ Injected msa_feat for 2O33_A | shape=torch.Size([26, 20])
📦 input keys: dict_keys(['token_index', 'residue_index', 'asym_id', 'entity_id', 'sym_id', 'restype', 'token_bonds', 'ref_pos', 'ref_mask', 'ref_element', 'ref_charge', 'ref_atom_name_chars', 'ref_space_uid', 'has_frame', 'frame_atom_index', 'atom_to_token_idx', 'atom_to_tokatom_idx', 'is_protein', 'is_ligand', 'is_dna', 'is_rna', 'resolution', 'mol_id', 'mol_atom_index', 'entity_mol_id', 'pae_rep_atom_mask', 'plddt_m_rep_atom_mask', 'distogram_rep_atom_mask', 'modified_res_mask', 'bond_mask', 'msa', 'has_deletion', 'deletion_value', 'profile', 'deletion_mean', 'prot_pair_num_alignments', 'prot_unpair_num_alignments', 'rna_pair_num_alignments', 'rna_unpair_num_alignments', 'template_restype', 'template_all_atom

 29%|██▉       | 243/844 [51:48<2:08:50, 12.86s/it]

🧪 Prediction shape: torch.Size([1, 20, 3]), contains NaN: False
✅ TM-score: 0.2889

▶ 243: 2IL9_M | len=142 | TM=None
✅ Loaded MSA for 2IL9_M from /kaggle/input/stanford-rna-3d-folding/MSA | shape=torch.Size([64, 142])
✅ Injected msa_feat for 2IL9_M | shape=torch.Size([64, 142])
📦 input keys: dict_keys(['token_index', 'residue_index', 'asym_id', 'entity_id', 'sym_id', 'restype', 'token_bonds', 'ref_pos', 'ref_mask', 'ref_element', 'ref_charge', 'ref_atom_name_chars', 'ref_space_uid', 'has_frame', 'frame_atom_index', 'atom_to_token_idx', 'atom_to_tokatom_idx', 'is_protein', 'is_ligand', 'is_dna', 'is_rna', 'resolution', 'mol_id', 'mol_atom_index', 'entity_mol_id', 'pae_rep_atom_mask', 'plddt_m_rep_atom_mask', 'distogram_rep_atom_mask', 'modified_res_mask', 'bond_mask', 'msa', 'has_deletion', 'deletion_value', 'profile', 'deletion_mean', 'prot_pair_num_alignments', 'prot_unpair_num_alignments', 'rna_pair_num_alignments', 'rna_unpair_num_alignments', 'template_restype', 'template_all_atom

 29%|██▉       | 244/844 [52:11<2:35:31, 15.55s/it]

🧪 Prediction shape: torch.Size([1, 142, 3]), contains NaN: False
✅ TM-score: 0.3457

▶ 244: 2IL9_A | len=142 | TM=None
✅ Loaded MSA for 2IL9_A from /kaggle/input/stanford-rna-3d-folding/MSA | shape=torch.Size([64, 142])
✅ Injected msa_feat for 2IL9_A | shape=torch.Size([64, 142])
📦 input keys: dict_keys(['token_index', 'residue_index', 'asym_id', 'entity_id', 'sym_id', 'restype', 'token_bonds', 'ref_pos', 'ref_mask', 'ref_element', 'ref_charge', 'ref_atom_name_chars', 'ref_space_uid', 'has_frame', 'frame_atom_index', 'atom_to_token_idx', 'atom_to_tokatom_idx', 'is_protein', 'is_ligand', 'is_dna', 'is_rna', 'resolution', 'mol_id', 'mol_atom_index', 'entity_mol_id', 'pae_rep_atom_mask', 'plddt_m_rep_atom_mask', 'distogram_rep_atom_mask', 'modified_res_mask', 'bond_mask', 'msa', 'has_deletion', 'deletion_value', 'profile', 'deletion_mean', 'prot_pair_num_alignments', 'prot_unpair_num_alignments', 'rna_pair_num_alignments', 'rna_unpair_num_alignments', 'template_restype', 'template_all_ato

In [ ]:
from scipy.ndimage import gaussian_filter1d

def smooth_coords(df, sigma=1.0):
    df = df.copy()
    for n in range(1, 6):  # 5개의 구조 (x_1 ~ x_5, y_1 ~ y_5, z_1 ~ z_5)
        for axis in ['x', 'y', 'z']:
            col = f"{axis}_{n}"
            if col in df.columns:
                df[col] = gaussian_filter1d(df[col].values, sigma=sigma)
    return df

In [ ]:
if MODEL_TYPE=='protenix' and not VALIDATION:
    test_df=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')
    import warnings
    warnings.filterwarnings("ignore")  
    
    dataset = DictDataset(test_df.sequence, dump_dir='output', id_list=test_df.target_id, use_msa_server=True)
    num_data = len(dataset)
    for i, seq in tqdm(enumerate(test_df.sequence),total=num_data):
        try:
            data, atom_array, data_error_message=dataset[i]
            target_id = data["sample_name"]
            assert target_id==test_df.target_id[i]
            assert data_error_message==''
            
            new_configs = update_inference_configs(configs, data["N_token"].item())
            runner.update_model_configs(new_configs)
            prediction = runner.predict(data)
            prediction=prediction['coordinate'][:,data['input_feature_dict']['atom_to_tokatom_idx']==12]

            result = parse_output_to_df(prediction, seq, target_id)[0]
        except:
            target_id==test_df.target_id[i]
            print('Failed to predict', target_id)
            result=pd.DataFrame(columns=['ID', 'resname', 'resid', 
                                         'x_1', 'y_1', 'z_1', 
                                         'x_2', 'y_2', 'z_2',
                                         'x_3', 'y_3', 'z_3', 
                                         'x_4', 'y_4', 'z_4', 
                                         'x_5', 'y_5', 'z_5'], 
                                         data=[[target_id, x, j+1] + [0.0]*15 for j, x in enumerate(seq)])
        result = smooth_coords(result, sigma=1.0)
        result['ID']=result.apply(lambda x: x.ID + '_' + str(x.resid), axis=1)
        result.to_csv('submission.csv', index=False, mode='a', header=(i==0))
        torch.cuda.empty_cache()

    display(pd.read_csv('submission.csv'))